<a href="https://colab.research.google.com/github/aycaecemgul/InceptionV3_COVID-19/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import os
import pickle
import random

import numpy as np
import tensorflow as tf
import tensorflow.keras
from PIL import Image
from matplotlib import pyplot as plt
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from numpy import asarray
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from numpy import asarray
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive

drive.mount(‘/content/gdrive’)

In [ ]:
!unrar x "/content/drive/My Drive/Desktop.rar" "/content/drive/My Drive/"

In [ ]:
DIR = 'D:\Tez\Datasets\kaggle'
CATEGORIES = ["0NonCOVID","1COVID","2CAP"]
IMG_SIZE = 150
IMG_SHAPE=(150,150,3)

In [ ]:
dataset_train=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(IMG_SIZE,IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="training"
)

dataset_val=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(IMG_SIZE,IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="validation"
)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
dataset_train = dataset_train.map(lambda x, y: (normalization_layer(x), y))
dataset_val = dataset_val.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
base_model = InceptionV3(
                          weights='imagenet',
                          include_top=False,
                          input_shape=IMG_SHAPE
                        )

base_model.trainable=False

model = tf.keras.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(dataset_train,batch_size=32,validation_data = dataset_val,epochs = 5,verbose=2)

In [ ]:
model.save("covid-19.model")